In [2]:
import numpy as np
from scipy import sparse
import qutip
import itertools

I = sparse.eye(2, format='csc', dtype=complex)
X = sparse.csc_matrix(([1, 1], ([0, 1], [1, 0])), shape=(2, 2), dtype=complex)
Y = sparse.csc_matrix(([-1j, 1j], ([0, 1], [1, 0])), shape=(2, 2), dtype=complex)
Z = sparse.csc_matrix(([1, -1], ([0, 1], [0, 1])), shape=(2, 2), dtype=complex)

def op_on_subset(A, subset, n):
    """ Return an operator acting on a subset of qubits.

    The method used is slightly convoluted and is intended to minimize the
    number of calls to scipy.sparse.kron.

    Args: 
        A: the matrix (should be 2x2)
        subset: tuple of indices of the qubits to act on
        n: the total number of qubits
    Output:
        M: a 2^n x 2^n sparse matrix
    """
    # Handle empty subset
    if len(subset) == 0:
        return sparse.eye(2 ** n, format='csc')

    # construct list of (length, indicator) tuples
    subset = sorted(subset)
    # The variable below will store the length of each group of qubits along with
    # an indicator of whether that group is acted on or not
    lengths = []
    i = subset[0]           # index representing beginning of active group
    if i != 0:
        lengths.append((i, 0))
    k = 1                   # index of current element of subset being considered
    while k < len(subset):
        if subset[k] == subset[k-1] + 1:
            k += 1
        else:
            lengths.append((subset[k-1] - i + 1, 1))
            lengths.append((subset[k] - subset[k-1] - 1, 0))
            i = subset[k]
            k += 1
    lengths.append((subset[k-1] - i + 1, 1))
    if subset[k-1] != n - 1:
        lengths.append((n - subset[k-1] - 1, 0))

    # construct the operator from the list of (length, indicator) tuples
    if lengths[0][1] == 0:
        M = sparse.eye(2**lengths[0][0])
    else:
        M = A
        for l in range(1, lengths[0][0]):
            M = sparse.kron(M, A, format='csc')

    for m in range(1, len(lengths)):
        if lengths[m][1] == 0:
            M = sparse.kron(M, sparse.eye(2**lengths[m][0]), format='csc')
        else:
            N = A
            for l in range(1, lengths[m][0]):
                N = sparse.kron(N, A, format='csc')
            M = sparse.kron(M, N, format='csc')
            del N

    return M

def swap_ij(i, j, n):
    """
    Return the swap operator acting on qubits i and j.
    
    Args: 
        i, j: the qubits to act on
        n: the number of qubits
    """
    return .5 * (sparse.eye(2**n, format='csc') + op_on_subset(X, [i, j], n) +
                 op_on_subset(Y, [i, j], n) + op_on_subset(Z, [i, j], n))

def cz_ij(i, j, n):
    """
    Return the controlled-Z operator acting on qubits i and j.
    
    Args:
        i, j: the qubits to act on
        n: the number of qubits
    """
    return .5 * (sparse.eye(2**n, format='csc') + op_on_subset(Z, [i], n) +
                 op_on_subset(Z, [j], n) - op_on_subset(Z, [i, j], n))

def cnot_ij(i, j, n):
    """
    Return the controlled-X operator acting on qubits i and j.
    
    Args:
        i, j: the qubits to act on
        n: the number of qubits
    """
    return .5 * (sparse.eye(2**n, format='csc') +
                 op_on_subset(X, [j], n) +
                 op_on_subset(Z, [i], n) -
                 op_on_subset(Z, [i], n) * op_on_subset(X, [j], n))

def bare_hopping_term(i, j, n):
    return .5 * (op_on_subset(X, [i], n) * op_on_subset(Y, [j], n) -
                 op_on_subset(Y, [i], n) * op_on_subset(X, [j], n))

def hopping_term(i, j, n):
    return bare_hopping_term(i, j, n) * op_on_subset(Z, range(i + 1, j), n)

def computational_basis_vector(occupied_orbitals, n_orbitals):
    one_index = sum([2 ** (n_orbitals - 1 - i) for i in occupied_orbitals])
    basis_vec = scipy.sparse.csc_matrix(([1.], ([one_index], [0])),
                                        shape=(2 ** n_orbitals, 1),
                                        dtype=float)
    return basis_vec

In [3]:
# Functions for implementation of Gamma

gate_function = {'cnot': cnot_ij,
                 'cz': cz_ij,
                 'swap': swap_ij}

def get_operator(circuit, n):
    """Return the matrix corresponding to a list of gates"""
    operator = sparse.eye(2 ** n, format='csc', dtype=complex)
    for gate, i, j in circuit:
        gate_func = gate_function[gate]
        operator = gate_func(i, j, n) * operator
    return operator

def row_qubits(k, n):
    """Return the qubits in row k where there are n qubits per row."""
    qubits = range(n * k, n * k + n)
    if k % 2 != 0:
        qubits = reversed(qubits)
    return list(qubits)

def parity_basis_change(n, sys_qubits):
    gates = []
    for row in reversed(range(1, n)):
        lower_row = row_qubits(row, n)
        upper_row = row_qubits(row - 1, n)
        for col in range(n):
            gates.append(('cnot',
                          sys_qubits[lower_row[col]],
                          sys_qubits[upper_row[col]]))
    return gates

def leftward_cz(i, n, sys_qubits, anc_qubits):
    gates = []
    for row in range(2, n, 2):
        anc_qubit = anc_qubits[row]
        sys_qubit_1 = sys_qubits[row_qubits(row - 2, n)[n - 1 - i]]
        sys_qubit_2 = sys_qubits[row_qubits(row, n)[n - 1 - i]]
        gates.append(('cz', anc_qubit, sys_qubit_1))
        gates.append(('cz', anc_qubit, sys_qubit_2))
    return gates

def leftward_swap(i, n, sys_qubits, anc_qubits):
    gates = []
    for row in range(n):
        anc_qubit = anc_qubits[row]
        sys_qubit = sys_qubits[row_qubits(row, n)[n - 1 - i]]
        gates.append(('swap', anc_qubit, sys_qubit))
        gates.append(('cnot', anc_qubit, sys_qubit))
        anc_qubits[row] = sys_qubit
        sys_qubits[row_qubits(row, n)[n - 1 - i]] = anc_qubit
    return gates

def rightward_swap_and_cz(i, n, sys_qubits, anc_qubits):
    gates = []
    for j in range(0, n, 2):
        upper_row = j
        upper_anc_qubit = anc_qubits[upper_row]
        upper_sys_qubit = sys_qubits[row_qubits(upper_row, n)[i]]
        if j < n - 1:
            lower_row = j + 1
            lower_anc_qubit = anc_qubits[lower_row]
            lower_sys_qubit = sys_qubits[row_qubits(lower_row, n)[i]]
        gates.append(('cnot', upper_sys_qubit, upper_anc_qubit))
        gates.append(('cz', upper_anc_qubit, upper_sys_qubit))
        if j < n - 1:
            gates.append(('cnot', lower_sys_qubit, lower_anc_qubit))
            gates.append(('cz', lower_anc_qubit, upper_sys_qubit))
        gates.append(('swap', upper_anc_qubit, upper_sys_qubit))
        anc_qubits[upper_row] = upper_sys_qubit
        sys_qubits[row_qubits(upper_row, n)[i]] = upper_anc_qubit
        if j < n - 1:
            gates.append(('swap', lower_anc_qubit, lower_sys_qubit))
            anc_qubits[lower_row] = lower_sys_qubit
            sys_qubits[row_qubits(lower_row, n)[i]] = lower_anc_qubit
    return gates

def vertical_edges(n):
    edges = []
    for row in range(n - 1):
        upper_row = row_qubits(row, n)
        lower_row = row_qubits(row + 1, n)
        edges += zip(upper_row, lower_row)
    return edges

In [3]:
n = 5
N = n ** 2
m = N + n

sys_qubits = {}
anc_qubits = {}
for i in range(N):
    sys_qubits[i] = i
for i in range(n):
    anc_qubits[i] = m - n + i

stage_1 = parity_basis_change(n, sys_qubits)
for i in reversed(range(1, n)):
    stage_1.append(('cnot', anc_qubits[i], anc_qubits[i - 1]))

stage_2 = []
for i in range(n):
    stage_2 += leftward_cz(i, n, sys_qubits, anc_qubits)
    stage_2 += leftward_swap(i, n, sys_qubits, anc_qubits)
    
stage_3 = list(reversed(parity_basis_change(n, sys_qubits)))
for i in range(n - 1):
    stage_3.append(('cnot', anc_qubits[i + 1], anc_qubits[i]))

stage_4 = []
for i in range(n):
    stage_4 += rightward_swap_and_cz(i, n, sys_qubits, anc_qubits)
    
print('Stage 1:')
for gate in stage_1:
    print('    {}'.format(gate))
print('Stage 2:')
for gate in stage_2:
    print('    {}'.format(gate))
print('Stage 3:')
for gate in stage_3:
    print('    {}'.format(gate))
print('Stage 4:')
for gate in stage_4:
    print('    {}'.format(gate))

Stage 1:
    ('cnot', 20, 19)
    ('cnot', 21, 18)
    ('cnot', 22, 17)
    ('cnot', 23, 16)
    ('cnot', 24, 15)
    ('cnot', 19, 10)
    ('cnot', 18, 11)
    ('cnot', 17, 12)
    ('cnot', 16, 13)
    ('cnot', 15, 14)
    ('cnot', 10, 9)
    ('cnot', 11, 8)
    ('cnot', 12, 7)
    ('cnot', 13, 6)
    ('cnot', 14, 5)
    ('cnot', 9, 0)
    ('cnot', 8, 1)
    ('cnot', 7, 2)
    ('cnot', 6, 3)
    ('cnot', 5, 4)
    ('cnot', 29, 28)
    ('cnot', 28, 27)
    ('cnot', 27, 26)
    ('cnot', 26, 25)
Stage 2:
    ('cz', 27, 4)
    ('cz', 27, 14)
    ('cz', 29, 14)
    ('cz', 29, 24)
    ('swap', 25, 4)
    ('cnot', 25, 4)
    ('swap', 26, 5)
    ('cnot', 26, 5)
    ('swap', 27, 14)
    ('cnot', 27, 14)
    ('swap', 28, 15)
    ('cnot', 28, 15)
    ('swap', 29, 24)
    ('cnot', 29, 24)
    ('cz', 14, 3)
    ('cz', 14, 13)
    ('cz', 24, 13)
    ('cz', 24, 23)
    ('swap', 4, 3)
    ('cnot', 4, 3)
    ('swap', 5, 6)
    ('cnot', 5, 6)
    ('swap', 14, 13)
    ('cnot', 14, 13)
    ('swap', 15, 16

In [4]:
# IF GAMMA IS CORRECT THE OUTPUT OF THIS CELL SHOULD BE ZERO

gamma_circuit = stage_1 + stage_2 + stage_3 + stage_4

gamma = get_operator(gamma_circuit, m)

discrepancy = 0.

for i, j in vertical_edges(n):
    A = hopping_term(i, j, m)
    B = gamma.T.conj() * bare_hopping_term(i, j, m) * gamma
    C = A - B

    anc_zero_indices = [z * 2 ** n for z in range(2 ** N)]
    D = C[anc_zero_indices][:, anc_zero_indices]

    if D.nnz:
        discrepancy = max(abs(D.data))
        
print(discrepancy)

MemoryError: 

In [4]:
# Test that removing CNOTS results in identity

stage_1_no_cz = [gate for gate in stage_1 if gate[0] != 'cz']
stage_2_no_cz = [gate for gate in stage_2 if gate[0] != 'cz']
stage_3_no_cz = [gate for gate in stage_3 if gate[0] != 'cz']
stage_4_no_cz = [gate for gate in stage_4 if gate[0] != 'cz']

print('Stage 1:')
for gate in stage_1_no_cz:
    print('    {}'.format(gate))
print('Stage 2:')
for gate in stage_2_no_cz:
    print('    {}'.format(gate))
print('Stage 3:')
for gate in stage_3_no_cz:
    print('    {}'.format(gate))
print('Stage 4:')
for gate in stage_4_no_cz:
    print('    {}'.format(gate))
print()

gamma_circuit = stage_1_no_cz + stage_2_no_cz + stage_3_no_cz + stage_4_no_cz
        
gamma = get_operator(gamma_circuit, m)
identity = sparse.eye(2 ** m)

difference = gamma - identity
discrepancy = 0.
if difference.nnz:
    discrepancy = max(abs(difference.data))
print(discrepancy)

Stage 1:
    ('cnot', 15, 8)
    ('cnot', 14, 9)
    ('cnot', 13, 10)
    ('cnot', 12, 11)
    ('cnot', 8, 7)
    ('cnot', 9, 6)
    ('cnot', 10, 5)
    ('cnot', 11, 4)
    ('cnot', 7, 0)
    ('cnot', 6, 1)
    ('cnot', 5, 2)
    ('cnot', 4, 3)
    ('cnot', 19, 18)
    ('cnot', 18, 17)
    ('cnot', 17, 16)
Stage 2:
    ('cnot', 3, 16)
    ('cnot', 4, 17)
    ('cnot', 11, 18)
    ('cnot', 12, 19)
    ('cnot', 2, 16)
    ('cnot', 5, 17)
    ('cnot', 10, 18)
    ('cnot', 13, 19)
    ('cnot', 1, 16)
    ('cnot', 6, 17)
    ('cnot', 9, 18)
    ('cnot', 14, 19)
    ('cnot', 0, 16)
    ('cnot', 7, 17)
    ('cnot', 8, 18)
    ('cnot', 15, 19)
Stage 3:
    ('cnot', 4, 3)
    ('cnot', 5, 2)
    ('cnot', 6, 1)
    ('cnot', 7, 0)
    ('cnot', 11, 4)
    ('cnot', 10, 5)
    ('cnot', 9, 6)
    ('cnot', 8, 7)
    ('cnot', 12, 11)
    ('cnot', 13, 10)
    ('cnot', 14, 9)
    ('cnot', 15, 8)
    ('cnot', 17, 16)
    ('cnot', 18, 17)
    ('cnot', 19, 18)
Stage 4:
    ('cnot', 7, 17)
    ('cnot', 0, 16)


In [69]:
# Testing cell
for key, val in sys_qubits.items():
    print(key, val)
print()
for key, val in anc_qubits.items():
    print(key, val)
print()

for i in range(n):
    leftward_swap(i, n, sys_qubits, anc_qubits)
    
for key, val in sys_qubits.items():
    print(key, val)
print()
for key, val in anc_qubits.items():
    print(key, val)
print()

for i in range(n):
    rightward_swap_and_cz(i, n, sys_qubits, anc_qubits)
    
for key, val in sys_qubits.items():
    print(key, val)
print()
for key, val in anc_qubits.items():
    print(key, val)
print()

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15

0 16
1 17
2 18
3 19

0 1
1 2
2 3
3 16
4 17
5 4
6 5
7 6
8 9
9 10
10 11
11 18
12 19
13 12
14 13
15 14

0 0
1 7
2 8
3 15

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15

0 16
1 17
2 18
3 19



In [42]:
# Testing cell
gates = leftward_cz(1, n, sys_qubits, anc_qubits)
for gate in gates:
    print(gate)

('cz', 18, 2)
('cz', 18, 10)


In [27]:
# Testing cell
pbc = parity_basis_change(4, sys_qubits)
for gate in pbc:
    print(gate)

('cnot', 15, 8)
('cnot', 14, 9)
('cnot', 13, 10)
('cnot', 12, 11)
('cnot', 8, 7)
('cnot', 9, 6)
('cnot', 10, 5)
('cnot', 11, 4)
('cnot', 7, 0)
('cnot', 6, 1)
('cnot', 5, 2)
('cnot', 4, 3)


In [2]:
# Testing cell

n = 9

A = cnot_ij(1, 6, n) * cz_ij(1, 6, n) * cz_ij(4, 6, n) * cnot_ij(1, 6, n)
B = cz_ij(1, 6, n) * cz_ij(4, 6, n) * cz_ij(1, 4, n) * op_on_subset(Z, (1,), n)

difference = A - B
discrepancy = 0.
if difference.nnz:
    discrepancy = max(abs(difference.data))
print(discrepancy)

0.0


In [5]:
# Testing cell

n = 20

A = hopping_term(6, 9, n) * hopping_term(5, 10, n)
B = bare_hopping_term(6, 9, n) * bare_hopping_term(5, 10, n) * op_on_subset(Z, [6, 9], n)

difference = A - B
discrepancy = 0.
if difference.nnz:
    discrepancy = max(abs(difference.data))
print(discrepancy)

0.0


In [2]:
# Testing cell
A = .5 * (op_on_subset(X, [0], 2) * op_on_subset(Y, [1], 2) -
          op_on_subset(Y, [0], 2) * op_on_subset(X, [1], 2))
print(A.toarray())

[[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+1.j  0.+0.j]
 [ 0.+0.j  0.-1.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]]


In [14]:
# Testing cell
A = cnot_ij(0, 1, 3)
print(A.toarray())

[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j]]
